In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Thyroid_Cancer"
cohort = "GSE138198"

# Input paths
in_trait_dir = "../../input/GEO/Thyroid_Cancer"
in_cohort_dir = "../../input/GEO/Thyroid_Cancer/GSE138198"

# Output paths
out_data_file = "../../output/preprocess/Thyroid_Cancer/GSE138198.csv"
out_gene_data_file = "../../output/preprocess/Thyroid_Cancer/gene_data/GSE138198.csv"
out_clinical_data_file = "../../output/preprocess/Thyroid_Cancer/clinical_data/GSE138198.csv"
json_path = "../../output/preprocess/Thyroid_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Differentially expression profiling in Hashimoto's thyroiditis (HT), papillary thyroid carcinoma (PTC) with HT in background, PTC without HT in background, micro PTC (mPTC), and three normal thyroid samples (TN)."
!Series_summary	"Experiment: Establishment of expression profiles in HT, PTC with HT, PTC without HT, and mPTC in comparison to TN samples. TN samples were downloaded as CEL files from the repository of the microarray vendor. Biostatistical analysis focussed in first instance on identifying genes and biofunctions related to HT and PTC with HT."
!Series_overall_design	"We extracted RNA from tumor specimens and RNA with an RIN > 5 (Bioanalyzer, Agilent) or, in exceptional cases, RNA assessed by gel electrophoresis, was processed using the recommended processing kits and hybridized to Affymetrix Human Gene 1.0 ST arrays. Three normal tissue samples were downloaded as CEL files from the data resource center of the vendor (https://www.thermof

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# From the background information, this dataset appears to be microarray gene expression data
# (Affymetrix Human Gene 1.0 ST arrays) so we can set is_gene_available to True
is_gene_available = True  

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait: The information about thyroid cancer status is in row 1 (sample type)
trait_row = 1

# For age: No age information is available
age_row = None

# For gender: Gender information is available in row 0
gender_row = 0

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait value to binary (0: healthy, 1: thyroid cancer)"""
    if not value or 'not available' in value:
        return None
    
    # If the format contains a colon, extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'normal thyroid' in value.lower() or 'tn' in value.lower():
        return 0  # Normal/healthy thyroid
    elif any(cancer_term in value.lower() for cancer_term in ['ptc', 'carcinoma', 'mptc', 'micro ptc']):
        return 1  # Any type of thyroid cancer
    else:
        # For Hashimoto's thyroiditis without cancer, we'll return None as it's not clearly cancer or normal
        if "hashimoto's thyroiditis" in value.lower() and not "ptc" in value.lower():
            return None
        return None  # For any other unclear values

def convert_gender(value):
    """Convert gender value to binary (0: female, 1: male)"""
    if not value or 'not available' in value:
        return None
    
    # If the format contains a colon, extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.upper() == 'F':
        return 0  # Female
    elif value.upper() == 'M':
        return 1  # Male
    else:
        return None  # Unknown or not available

# There's no age data, but we'll define the function anyway in case it's needed
def convert_age(value):
    """Convert age value to continuous"""
    if not value or 'not available' in value:
        return None
    
    # If the format contains a colon, extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

# 3. Save Metadata
# Trait data availability depends on whether trait_row is None
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we should extract clinical features
if trait_row is not None:
    # Use the geo_select_clinical_features function to extract clinical features
    clinical_features_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_features_df)
    print("Preview of clinical features:")
    print(preview)
    
    # Save the clinical features to CSV
    clinical_features_df.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")


Preview of clinical features:
{'GSM4101749': [0.0, 0.0], 'GSM4101750': [0.0, 0.0], 'GSM4101751': [0.0, 0.0], 'GSM4101752': [0.0, 0.0], 'GSM4101753': [0.0, 0.0], 'GSM4101754': [0.0, 0.0], 'GSM4101755': [0.0, 0.0], 'GSM4101756': [0.0, 0.0], 'GSM4101757': [0.0, 0.0], 'GSM4101758': [0.0, 0.0], 'GSM4101759': [0.0, 0.0], 'GSM4101760': [0.0, 0.0], 'GSM4101761': [0.0, 1.0], 'GSM4101762': [1.0, 0.0], 'GSM4101763': [1.0, 1.0], 'GSM4101764': [1.0, 0.0], 'GSM4101765': [1.0, 0.0], 'GSM4101766': [1.0, 1.0], 'GSM4101767': [1.0, 0.0], 'GSM4101768': [1.0, 0.0], 'GSM4101769': [1.0, 0.0], 'GSM4101770': [1.0, 1.0], 'GSM4101771': [1.0, 0.0], 'GSM4101772': [1.0, 0.0], 'GSM4101773': [1.0, 0.0], 'GSM4101774': [1.0, 1.0], 'GSM4101775': [1.0, 1.0], 'GSM4101776': [1.0, 0.0], 'GSM4101777': [1.0, 0.0], 'GSM4101778': [1.0, 0.0], 'GSM4101779': [1.0, 1.0], 'GSM4101780': [1.0, 0.0], 'GSM4101781': [1.0, 0.0], 'GSM4101782': [0.0, nan], 'GSM4101783': [0.0, nan], 'GSM4101784': [0.0, nan]}
Clinical features saved to ../../

### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
try:
    with gzip.open(matrix_file, 'rt') as file:
        for line in file:
            if "!series_matrix_table_begin" in line:
                found_marker = True
                break
    
    if found_marker:
        print("Found the matrix table marker in the file.")
    else:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        
    # Try to extract gene data from the matrix file
    gene_data = get_genetic_data(matrix_file)
    
    if gene_data.shape[0] == 0:
        print("Warning: Extracted gene data has 0 rows.")
        is_gene_available = False
    else:
        print(f"Gene data shape: {gene_data.shape}")
        # Print the first 20 gene/probe identifiers
        print("First 20 gene/probe identifiers:")
        print(gene_data.index[:20].tolist())
        
except Exception as e:
    print(f"Error extracting gene data: {e}")
    is_gene_available = False
    
    # Try to diagnose the file format
    print("Examining file content to diagnose the issue:")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            for i, line in enumerate(file):
                if i < 10:  # Print first 10 lines to diagnose
                    print(f"Line {i}: {line.strip()[:100]}...")  # Print first 100 chars of each line
                else:
                    break
    except Exception as e2:
        print(f"Error examining file: {e2}")

if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")


SOFT file: ../../input/GEO/Thyroid_Cancer/GSE138198/GSE138198_family.soft.gz
Matrix file: ../../input/GEO/Thyroid_Cancer/GSE138198/GSE138198_series_matrix.txt.gz
Found the matrix table marker in the file.
Gene data shape: (33297, 36)
First 20 gene/probe identifiers:
['7892501', '7892502', '7892503', '7892504', '7892505', '7892506', '7892507', '7892508', '7892509', '7892510', '7892511', '7892512', '7892513', '7892514', '7892515', '7892516', '7892517', '7892518', '7892519', '7892520']


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be probe IDs rather than human gene symbols.
# They look like Illumina or Affymetrix probe IDs (numeric IDs).
# We need to map these probe IDs to human gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Get a more complete view to understand the annotation structure
print("\nComplete sample of a few rows:")
print(gene_annotation.iloc[:3, :4].to_string())  # Show only first few columns for readability

# Examine the gene_assignment column which contains gene information
print("\nChecking for gene information in gene_assignment column:")
sample_text = gene_annotation.loc[0, 'gene_assignment'] if 'gene_assignment' in gene_annotation.columns else "Column not found"
print(f"Sample gene_assignment text: {sample_text[:200]}...")  # Print first 200 chars

# Extract gene symbols from gene_assignment column
if 'gene_assignment' in gene_annotation.columns:
    # Create a new column with extracted gene symbols
    gene_annotation['Gene'] = gene_annotation['gene_assignment'].apply(extract_human_gene_symbols)
    
    # Display a sample of the extracted gene symbols
    print("\nSample of extracted gene symbols:")
    sample_df = gene_annotation[['ID', 'Gene']].head(10)
    print(sample_df)
    
    # Check if gene symbols were successfully extracted
    empty_genes = gene_annotation['Gene'].apply(lambda x: len(x) == 0).sum()
    print(f"\nNumber of probes with no gene symbols extracted: {empty_genes} out of {len(gene_annotation)}")
    
    # Create the mapping using our new Gene column
    mapping_data = gene_annotation[['ID', 'Gene']].copy()
    mapping_data = mapping_data[mapping_data['Gene'].apply(len) > 0]  # Filter out empty gene lists
    
    # Print sample of the mapping to confirm
    print("\nSample of probe ID to gene symbol mappings:")
    print(mapping_data.head(10))
    
    # Check the size of the mapping data
    print(f"\nTotal number of probe-to-gene mappings: {len(mapping_data)}")
else:
    print("\nError: Could not find 'gene_assignment' column in the annotation data")



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_LIST', 'SPOT_ID', 'seqname', 'RANGE_GB', 'RANGE_STRAND', 'RANGE_START', 'RANGE_STOP', 'total_probes', 'gene_assignment', 'mrna_assignment', 'category']
{'ID': ['7896736', '7896738', '7896740', '7896742', '7896744'], 'GB_LIST': [nan, nan, 'NM_001004195,NM_001005240,NM_001005484,BC136848,BC136867,BC136907,BC136908', 'NR_024437,XM_006711854,XM_006726377,XR_430662,AK298283,AL137655,BC032332,BC118988,BC122537,BC131690,NM_207366,AK301928,BC071667', 'NM_001005221,NM_001005224,NM_001005277,NM_001005504,BC137547,BC137568'], 'SPOT_ID': ['chr1:53049-54936', 'chr1:63015-63887', 'chr1:69091-70008', 'chr1:334129-334296', 'chr1:367659-368597'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['53049', '63015', '69091', '334129', '367659'], 'RANGE_STOP': ['54936', '63887', '70008', '


Sample of extracted gene symbols:
        ID                                               Gene
0  7896736                                                 []
1  7896738                          [OR4G2P, OR4G11P, OR4G1P]
2  7896740  [OR4F4, OR4F17, OR4F5, BC136848, BC136867, BC1...
3  7896742  [F-, AK298283, AL137655, BC032332, PCMTD2, L-,...
4  7896744  [OR4F29, OR4F3, OR4F16, OR4F21, OR4F7P, OR4F1P...
5  7896746                                            [MT-TM]
6  7896748                                            [MT-TW]
7  7896750                                            [MT-TD]
8  7896752                                            [MT-TK]
9  7896754                               [AK026836, AK290103]

Number of probes with no gene symbols extracted: 1206898 out of 1232025



Sample of probe ID to gene symbol mappings:
         ID                                               Gene
1   7896738                          [OR4G2P, OR4G11P, OR4G1P]
2   7896740  [OR4F4, OR4F17, OR4F5, BC136848, BC136867, BC1...
3   7896742  [F-, AK298283, AL137655, BC032332, PCMTD2, L-,...
4   7896744  [OR4F29, OR4F3, OR4F16, OR4F21, OR4F7P, OR4F1P...
5   7896746                                            [MT-TM]
6   7896748                                            [MT-TW]
7   7896750                                            [MT-TD]
8   7896752                                            [MT-TK]
9   7896754                               [AK026836, AK290103]
10  7896756               [FAM87B, FAM87A, BC037297, BC118644]

Total number of probe-to-gene mappings: 25127


### Step 6: Gene Identifier Mapping

In [7]:
# Based on our analysis, we need to map the probe IDs from the gene expression matrix to gene symbols
# using the gene_assignment column from the annotation data

# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene data from matrix file
gene_data = get_genetic_data(matrix_file)
print(f"Gene expression data shape: {gene_data.shape}")
print(f"Sample of gene_data index (probe IDs): {gene_data.index[:5].tolist()}")

# 3. Get gene annotation data
gene_annotation = get_gene_annotation(soft_file)

if 'gene_assignment' in gene_annotation.columns:
    # Extract gene symbols from gene_assignment column
    gene_annotation['Gene'] = gene_annotation['gene_assignment'].apply(extract_human_gene_symbols)
    
    # Print detailed mapping info
    symbol_counts = gene_annotation['Gene'].apply(len).sum()
    print(f"Total gene symbols extracted: {symbol_counts}")
    print(f"Probes with at least one gene symbol: {(gene_annotation['Gene'].apply(len) > 0).sum()}")
    
    # Create mapping dataframe
    mapping_df = gene_annotation[['ID', 'Gene']].copy()
    # Filter out empty gene lists while preserving ID as string type
    mapping_df = mapping_df[mapping_df['Gene'].apply(len) > 0]
    mapping_df['ID'] = mapping_df['ID'].astype(str)
    
    # Check that gene_data index and mapping_df ID column are compatible
    print(f"Gene data index type: {type(gene_data.index[0])}")
    print(f"Mapping ID column type: {type(mapping_df['ID'].iloc[0])}")
    
    # Check if there are matching IDs between the two dataframes
    gene_ids_set = set(gene_data.index.astype(str))
    mapping_ids_set = set(mapping_df['ID'])
    intersection = gene_ids_set.intersection(mapping_ids_set)
    print(f"Number of matching IDs between gene data and mapping: {len(intersection)}")
    
    if len(intersection) > 0:
        # Apply gene mapping to convert probe-level data to gene expression data
        gene_data = apply_gene_mapping(gene_data, mapping_df)
        
        print(f"After mapping, gene expression data shape: {gene_data.shape}")
        print("First 10 gene symbols after mapping:")
        print(gene_data.index[:10].tolist() if len(gene_data.index) >= 10 else gene_data.index.tolist())
        
        # Preview a small portion of the gene expression data
        print("\nSample of gene expression data:")
        display_rows = min(5, len(gene_data))
        display_cols = min(3, len(gene_data.columns))
        if display_rows > 0:
            print(gene_data.iloc[:display_rows, :display_cols])
        else:
            print("No gene expression data available after mapping.")
            
        # If we successfully mapped genes, normalize gene symbols
        if len(gene_data) > 0:
            # Optional: normalize gene symbols to handle different versions of the same gene
            try:
                gene_data = normalize_gene_symbols_in_index(gene_data)
                print(f"\nAfter normalizing gene symbols, shape: {gene_data.shape}")
                print(f"Sample of normalized gene symbols: {gene_data.index[:10].tolist() if len(gene_data.index) >= 10 else gene_data.index.tolist()}")
            except Exception as e:
                print(f"Warning: Could not normalize gene symbols: {str(e)}")
    else:
        print("Error: No matching IDs between gene data and mapping data.")
        is_gene_available = False
else:
    print("Error: Could not find gene_assignment column for mapping")
    is_gene_available = False

# Save gene data if we have a valid result
if 'gene_data' in locals() and len(gene_data) > 0:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Saved gene expression data to {out_gene_data_file}")
else:
    print("Warning: No gene data was saved because mapping produced empty results.")
    is_gene_available = False


Gene expression data shape: (33297, 36)
Sample of gene_data index (probe IDs): ['7892501', '7892502', '7892503', '7892504', '7892505']


Total gene symbols extracted: 155269
Probes with at least one gene symbol: 25127


Gene data index type: <class 'str'>
Mapping ID column type: <class 'str'>
Number of matching IDs between gene data and mapping: 25127
After mapping, gene expression data shape: (0, 36)
First 10 gene symbols after mapping:
[]

Sample of gene expression data:
No gene expression data available after mapping.


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
try:
    # Make sure the directory exists
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Use the gene_data variable from the previous step (don't try to load it from file)
    print(f"Gene data shape before normalization: {gene_data.shape}")
    
    # Apply normalization to gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    # Use the normalized data for further processing
    gene_data = normalized_gene_data
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Load clinical data with correct row indices
try:
    # Load the clinical data from file
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    
    # Debug the trait and gender conversion functions
    print("Sample conversion results:")
    sample_trait = list(clinical_data.iloc[0])[1]  # Get a sample from trait row
    sample_gender = list(clinical_data.iloc[1])[1]  # Get a sample from gender row
    print(f"Sample trait value: '{sample_trait}' -> {convert_trait(sample_trait)}")
    print(f"Sample gender value: '{sample_gender}' -> {convert_gender(sample_gender)}")
    
    # Extract clinical features with CORRECT indices from Step 2
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=0,       # Correct trait row from Step 2
        convert_trait=convert_trait,
        gender_row=1,      # Correct gender row from Step 2
        convert_gender=convert_gender,
        age_row=None,      # Age row is None as per Step 2
        convert_age=None
    )
    
    print(f"Extracted clinical data shape: {clinical_features.shape}")
    print("Preview of clinical data (first 5 samples):")
    print(clinical_features.iloc[:, :5])
    
    # Save the properly extracted clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    is_trait_available = True
except Exception as e:
    print(f"Error extracting clinical data: {e}")
    is_trait_available = False

# 3. Link clinical and genetic data if both are available
if is_trait_available and is_gene_available:
    try:
        # Debug the column names to ensure they match
        print(f"Gene data columns (first 5): {gene_data.columns[:5].tolist()}")
        print(f"Clinical data columns (first 5): {clinical_features.columns[:5].tolist()}")
        
        # Check for common sample IDs
        common_samples = set(gene_data.columns).intersection(clinical_features.columns)
        print(f"Found {len(common_samples)} common samples between gene and clinical data")
        
        if len(common_samples) > 0:
            # Link the clinical and genetic data
            linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
            print(f"Initial linked data shape: {linked_data.shape}")
            
            # Debug the trait values before handling missing values
            print("Preview of linked data (first 5 rows, first 5 columns):")
            print(linked_data.iloc[:5, :5])
            
            # Handle missing values
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Linked data shape after handling missing values: {linked_data.shape}")
            
            if linked_data.shape[0] > 0:
                # Check for bias in trait and demographic features
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                
                # Validate the data quality and save cohort info
                note = "Dataset contains gene expression data from thyroid cancer samples with tumor type information."
                is_usable = validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=is_biased,
                    df=linked_data,
                    note=note
                )
                
                # Save the linked data if it's usable
                if is_usable:
                    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                    linked_data.to_csv(out_data_file)
                    print(f"Linked data saved to {out_data_file}")
                else:
                    print("Data not usable for the trait study - not saving final linked data.")
            else:
                print("After handling missing values, no samples remain.")
                validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=True,
                    df=pd.DataFrame(),
                    note="No valid samples after handling missing values."
                )
        else:
            print("No common samples found between gene expression and clinical data.")
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available,
                is_biased=True,
                df=pd.DataFrame(),
                note="No common samples between gene expression and clinical data."
            )
    except Exception as e:
        print(f"Error linking or processing data: {e}")
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=True,  # Assume biased if there's an error
            df=pd.DataFrame(),  # Empty dataframe for metadata
            note=f"Error in data processing: {str(e)}"
        )
else:
    # We can't proceed with linking if either trait or gene data is missing
    print("Cannot proceed with data linking due to missing trait or gene data.")
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=True,  # Data is unusable if we're missing components
        df=pd.DataFrame(),  # Empty dataframe for metadata
        note="Missing essential data components for linking (trait data or gene expression data)."
    )

Gene data shape before normalization: (0, 36)
Gene data shape after normalization: (0, 36)
Normalized gene data saved to ../../output/preprocess/Thyroid_Cancer/gene_data/GSE138198.csv
Sample conversion results:
Sample trait value: 'gender: F' -> None
Sample gender value: 'sample type: Hashimoto's thyroiditis (HT), papillary thyroid carcinoma (PTC) with HT in background, PTC without HT in background, micro PTC (mPTC), and three normal thyroid samples (TN).' -> None
Extracted clinical data shape: (2, 36)
Preview of clinical data (first 5 samples):
                GSM4101749  GSM4101750  GSM4101751  GSM4101752  GSM4101753
Thyroid_Cancer         NaN         NaN         NaN         NaN         NaN
Gender                 NaN         NaN         NaN         NaN         NaN
Clinical data saved to ../../output/preprocess/Thyroid_Cancer/clinical_data/GSE138198.csv
Gene data columns (first 5): ['GSM4101749', 'GSM4101750', 'GSM4101751', 'GSM4101752', 'GSM4101753']
Clinical data columns (first 5): 